In [3]:
# Import the necessary libraries
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



In [11]:
# Load the data file
file_path = ("/home/munyao/Desktop/flat_iron_school/linear-regression/Data/1000_Companies.csv")
df = pd.read_csv(file_path,index_col = 0)
df.dtypes()

TypeError: 'Series' object is not callable

In [1]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Evaluate the model's performance
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("RMSE: ", rmse)
print("R2 score: ", r2)

NameError: name 'train_test_split' is not defined